In [ ]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

## Stdlib
from collections import defaultdict, namedtuple
import math
import os
import tempfile

## Non-std libs
from matplotlib.colors import hsv_to_rgb
import matplotlib.pyplot as plt
import pandas as pd

## Local modules
from scn_rrd import rrd_meta_utils, rrd_utils, rrd_utils_stdout, plot_utils

### Data rate consumption

In [ ]:
locations = [
    'FCS',
    'nickelsville-cd',
    'northlake',
    'occ-oromo',
    'sps-franklin',
    'sps-garfield',
]
portGroups = [
    'epc-backhaul-interface',
    'backhaul-interface',
]

meta = rrd_meta_utils.read_meta()
meta = meta[
    ( meta['location'].isin(locations) ) &
    ( meta['port_group_name'].isin(portGroups) )
]

In [ ]:
portId_to_consumeDf = {
    row['port_id'] :
    rrd_utils.read_rrd_via_scp(row['hostname'], row['port_rrd_filename'], '-1month')
    for (_, row) in meta.iterrows()
}

In [ ]:
## Merge all ports for each device.
location_to_consumeDf = dict()
for (_, row) in meta.iterrows():
    (location, port_id) = row[[ 'location', 'port_id' ]]
    df = location_to_consumeDf.get(location, pd.DataFrame())
    port_df = portId_to_consumeDf[port_id]
    df = pd.concat([ df, port_df ], axis=0, ignore_index=True)
    location_to_consumeDf[location] = df

In [ ]:
(fig, ax) = plt.subplots(figsize=(12, 6))
for (location, consumeDf) in location_to_consumeDf.items():
    dt_sec = (consumeDf['time'][1] - consumeDf['time'][0]).seconds
    factor = dt_sec / pow(2, 30) # Raw byte/sec --> byte --> GB.

    aggr_df = (
        consumeDf[['time', 'INOCTETS', 'OUTOCTETS']]
        .groupby(by=pd.Grouper(key='time', freq='D'))
        .sum()
        * factor
    )
    
    color_h = plot_utils.LOC_TO_COLOR_H[location]
    aggr_df['INOCTETS'].plot(ax=ax, label=f"{location} downloads", color=hsv_to_rgb((color_h, 0.5, 1)))
    aggr_df['OUTOCTETS'].plot(ax=ax, label=f"{location} uploads", color=hsv_to_rgb((color_h, 1, 1)))
ax.set_title('Daily consumption')
ax.set_ylabel('Gigabyte')
ax.set_xlabel('Day')
ax.legend()
None # Hide stdout output of the above line

In [ ]:
(fig, ax) = plt.subplots(figsize=(12, 6))
for (location, consumeDf) in location_to_consumeDf.items():
    dt_sec = (consumeDf['time'][1] - consumeDf['time'][0]).seconds
    days = ( max(consumeDf['time']) - min(consumeDf['time']) ).days
    factor = dt_sec / pow(2, 30) / days # Raw byte/sec --> byte --> GB --> GB/day.

    aggr_df = (
        consumeDf[['time', 'INOCTETS', 'OUTOCTETS']]
        .groupby(consumeDf['time'].dt.hour)
        .sum(numeric_only=True)
        * factor
    )

    color_h = plot_utils.LOC_TO_COLOR_H[location]
    aggr_df['INOCTETS'].plot(ax=ax, label=f"{location} downloads", color=hsv_to_rgb((color_h, 0.5, 1)))
    aggr_df['OUTOCTETS'].plot(ax=ax, label=f"{location} uploads", color=hsv_to_rgb((color_h, 1, 1)))
ax.set_title('Average hourly consumption')
ax.set_ylabel('Gigabyte')
ax.set_xlabel('Hour')
ax.legend()
None # Hide stdout output of the above line

### Data rate capacity (aka speedtest)

In [ ]:
MonitorDevice = namedtuple('MonitorDevice', ['location', 'login_env_key'])
monitor_devices = [
    MonitorDevice('nickelsville-cd', 'CD_MON_LOGIN'),
    MonitorDevice('sps-franklin', 'FRANKLIN_MON_LOGIN'),
#     MonitorDevice('lihi-southend', 'SOUTHEND_MON_LOGIN'),
    MonitorDevice('FCS', 'FCS_MON_LOGIN'),
]
# We cannot get a reliable ssh session with the southend device. Neither the scp way nor the stdout way works.

In [ ]:
location_to_rrdName_to_capDf = defaultdict(dict)
for mondev in monitor_devices:
    login = rrd_meta_utils.DOTENV_ENTRIES[mondev.login_env_key]
    (addr, user, pw) = login.split('%%%%')
    
    for rrdName in ['down_rate', 'up_rate']:
        remote_rrd_filepath = f"~/speedtest/rrd/{rrdName}.rrd"

        df = rrd_utils_stdout.read_rrd_via_stdout(addr, user, pw, remote_rrd_filepath, '-2week')

        location_to_rrdName_to_capDf[mondev.location][rrdName] = df

In [ ]:
(fig, ax) = plt.subplots(figsize=(12, 6))
for mondev in monitor_devices:
    color_h = plot_utils.LOC_TO_COLOR_H[mondev.location]
    
    down = location_to_rrdName_to_capDf[mondev.location]['down_rate']
    ax.plot(down['time'], down['down_rate'], label=f"{mondev.location} download", color=hsv_to_rgb((color_h, 0.5, 1)))
    
    up = location_to_rrdName_to_capDf[mondev.location]['up_rate']
    ax.plot(up['time'], up['up_rate'], label=f"{mondev.location} upload", color=hsv_to_rgb((color_h, 1, 1)))
ax.set_title('Capacity')
ax.set_ylabel('Mbps')
ax.legend()
None # Hide stdout output of the above line

In [ ]:
def utilization(consume_df: pd.DataFrame, capacity_df: pd.DataFrame, factor: float) -> pd.DataFrame:
    '''
    @arg consume_df and capacity_df:
        Each DF should have two columns,
        1st column containing time, and
        2nd column containing quantity (float).
    @return:
        1st column contains time, unmodified from @arg consume_df.
        2nd column contains (consumed quantity / capacity quantity),
            where capacity quantity comes from the last measurement at or before the time.
    '''
    cap_i = 0
    cap_n = len(capacity_df)
    cap_quant_prev = None
    
    utilizn_rows = []
    
    for (_, (consu_time, consu_quant)) in consume_df.iterrows():
        if pd.isna(consu_quant):
            continue
        
        while cap_i < cap_n and capacity_df.iloc[cap_i][0] <= consu_time:
            cap_quant_prev = capacity_df.iloc[cap_i][1]
            cap_i += 1

        if pd.isna(cap_quant_prev):
            # No capacity was recorded before the current consumption timestamp.
            continue

        utilizn = consu_quant / cap_quant_prev * factor
        utilizn_row = (consu_time, utilizn)
        utilizn_rows.append(utilizn_row)

    return pd.DataFrame(utilizn_rows, columns=['time', 'utilization'])

In [ ]:
(fig, ax) = plt.subplots(figsize=(12, 6))

# Scale both numerator and denominator to bitsPerSec. Then scale to percentage.
factor = (1/8) / pow(2,20) * 100

for mondev in monitor_devices:
    color_h = plot_utils.LOC_TO_COLOR_H[mondev.location]
    
    consume_bytesPerSec_df = location_to_consumeDf[mondev.location][['time', 'INOCTETS']]
    cap_MegabitsPerSec_df = location_to_rrdName_to_capDf[mondev.location]['down_rate']
    utilzn_df = utilization(consume_bytesPerSec_df, cap_MegabitsPerSec_df, factor)
    ax.plot(utilzn_df['time'], utilzn_df['utilization'], label=f"{mondev.location} download", color=hsv_to_rgb((color_h, 0.5, 1)))
    
    consume_bytesPerSec_df = location_to_consumeDf[mondev.location][['time', 'OUTOCTETS']]
    cap_MegabitsPerSec_df = location_to_rrdName_to_capDf[mondev.location]['up_rate']
    utilzn_df = utilization(consume_bytesPerSec_df, cap_MegabitsPerSec_df, factor)
    ax.plot(utilzn_df['time'], utilzn_df['utilization'], label=f"{mondev.location} upload", color=hsv_to_rgb((color_h, 1, 1)))
ax.set_title('Utilization')
ax.set_ylabel('Percent')
ax.legend()
None # Hide stdout output of the above line